# Welding Experiment Design

In [ ]:
# if the package is not installed in your python environment, run this to execute the notebook directly from inside the GitHub repository
%cd -q ..

In [ ]:
# enable interactive plots on Jupyterlab with ipympl and jupyterlab-matplotlib installed
# %matplotlib widget

In [ ]:
# some python imports that will be used throughout the tutorial
import numpy as np

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import pint
import xarray as xr
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# importing the weldx package with prevalent default abbreviations
import weldx
import weldx.geometry as geo
import weldx.transformations as tf
import weldx.utility as ut
import weldx.visualization as vis
from weldx import Q_
from weldx.transformations import LocalCoordinateSystem as lcs
from weldx.transformations import WXRotation
from weldx.welding.groove.iso_9692_1 import get_groove

## helper functions

In [ ]:
time = pd.timedelta_range("0s","10s",freq="100ms")

In [ ]:
def sine_lcs(time, f=Q_(1,"Hz"), amp=Q_([[0,1,0]],"mm"), bias=None):
    """Create an interpolated sine LCS"""
    ts_sine = ut._sine(f=f,amp=amp, bias=bias)
    ts_sine_data = ts_sine.interp_time(time = time)
    # assign vector coordinates and convert to m
    ts_sine_data = ts_sine_data.rename({"dim_1": "c"}).assign_coords({"c": ["x", "y", "z"]})
    ts_sine_data.data = ts_sine_data.data.magnitude
    ts_sine_data["time"] = pd.TimedeltaIndex(ts_sine_data["time"].data)
    return lcs(coordinates=ts_sine_data)

In [ ]:
def distance(ptc,lcs_name,time=time):
    """Calcualate minimal distance between pointcloud and 3D trace"""
    
    lcs_interp = csm.get_cs(coordinate_system_name=lcs_name,reference_system_name="workpiece",time=time)
    
    trace = lcs_interp.coordinates.data
    
    ptc = ptc.T
    
    ptc = np.expand_dims(ptc,1)
    trace = np.expand_dims(trace,0)
    
    return np.min(np.sqrt(np.sum((ptc-trace)**2,axis=-1)),axis=0)

## Trace and CSM setup

In [ ]:
# define the weld seam length in mm
seam_length = Q_(100, "mm")

trace_segment = geo.LinearHorizontalTraceSegment(seam_length)
trace = geo.Trace(trace_segment)

In [ ]:
# crete a new coordinate system manager with default base coordinate system
csm = weldx.transformations.CoordinateSystemManager("base")

In [ ]:
# add the workpiece coordinate system
csm.add_cs(
    coordinate_system_name="workpiece",
    reference_system_name="base",
    lcs=trace.coordinate_system,
)

In [ ]:
tcp_start_point = Q_([0.0, 0.0, 0.0], "mm")
tcp_end_point = np.append(seam_length, Q_([0, 0], "mm"))

v_weld = Q_(10, "mm/s")
s_weld = (tcp_end_point - tcp_start_point)[0]  # length of the weld
t_weld = s_weld / v_weld

t_start = pd.Timedelta("0s")
t_end = pd.Timedelta(str(t_weld.to_base_units()))

coords = [tcp_start_point.magnitude, tcp_end_point.magnitude]

tcp_wire = lcs(coordinates=coords, time=[t_start, t_end])

# add the workpiece coordinate system
csm.add_cs(
    coordinate_system_name="tcp_wire",
    reference_system_name="workpiece",
    lcs=tcp_wire,
)

### add y weaving

In [ ]:
csm.add_cs(coordinate_system_name="tcp_sine_y", reference_system_name="tcp_wire", lcs=sine_lcs(time,f=Q_(1,"Hz"), amp=Q_([[0,1,0]],"mm"), bias=None))

## add z weaving

In [ ]:
csm.add_cs(coordinate_system_name="tcp_sine_z", reference_system_name="tcp_wire", lcs=sine_lcs(time,f=Q_(1,"Hz"), amp=Q_([[0,0,2]],"mm"),bias=Q_([0,0,2],"mm")))

In [ ]:
csm

## generate I-Groove pointcloud

In [ ]:
groove_1 = get_groove(
    groove_type="IGroove",
    workpiece_thickness=Q_(5, "mm"),
    root_gap=Q_(2, "mm"),
)

groove_2 = get_groove(
    groove_type="IGroove",
    workpiece_thickness=Q_(5, "mm"),
    root_gap=Q_(12, "mm"),
)

v_profile = geo.VariableProfile([groove_1.to_profile(), groove_2.to_profile()],
                            [0, 100],
                            [geo.linear_profile_interpolation_sbs]
                            )

# create 3d workpiece geometry from the groove profile and trace objects
geometry = geo.Geometry(profile=v_profile, trace=trace)
pointcloud_I = geometry.rasterize(profile_raster_width=Q_(1, "mm"),
                 trace_raster_width=Q_(1, "mm"))

## Calculate distance (simple trace)

In [ ]:
d = distance(pointcloud_I,"tcp_sine_y")
plt.plot(d)

## Generate V-Groove geometry

In [ ]:
groove_1 = get_groove(
    groove_type="VGroove",
    workpiece_thickness=Q_(5, "mm"),
    groove_angle=Q_(60, "deg"),
    root_face=Q_(1, "mm"),
    root_gap=Q_(1, "mm"),
)

groove_2 = get_groove(
    groove_type="VGroove",
    workpiece_thickness=Q_(5, "mm"),
    groove_angle=Q_(20, "deg"),
    root_face=Q_(1.5, "mm"),
    root_gap=Q_(1, "mm"),
)

v_profile = geo.VariableProfile([groove_1.to_profile(), groove_2.to_profile()],
                            [0, 100],
                            [geo.linear_profile_interpolation_sbs]
                            )

# create 3d workpiece geometry from the groove profile and trace objects
geometry = geo.Geometry(profile=v_profile, trace=trace)
pointcloud_V = geometry.rasterize(profile_raster_width=Q_(0.1, "mm"),
                 trace_raster_width=Q_(0.5, "mm"))

In [ ]:
d = distance(pointcloud_V,"tcp_sine_z")
plt.plot(d)